## Deliverable 2. Create a Customer Travel Destinations Map.

In [16]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [17]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Emba,KZ,48.8264,58.1442,75.79,47,34,6.69,scattered clouds
1,1,Tara,RU,56.8944,74.3710,75.60,59,90,7.70,overcast clouds
2,2,Rikitea,PF,-23.1203,-134.9692,72.90,74,12,16.42,light rain
3,3,Cabo San Lucas,MX,22.8909,-109.9124,86.61,61,41,3.00,scattered clouds
4,4,Upernavik,GL,72.7868,-56.1549,45.72,77,100,5.06,light rain


In [18]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 75


In [19]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Rikitea,PF,-23.1203,-134.9692,72.90,74,12,16.42,light rain
8,8,Jumla,NP,29.2747,82.1838,69.17,82,84,5.23,broken clouds
9,9,Vila Velha,BR,-20.3297,-40.2925,69.75,68,100,9.22,overcast clouds
17,17,Guerrero Negro,MX,27.9769,-114.0611,70.54,77,5,10.80,clear sky
19,19,Fairbanks,US,64.8378,-147.7164,71.60,58,75,14.97,broken clouds
27,27,Isangel,VU,-19.5500,169.2667,74.57,82,100,7.05,heavy intensity rain
30,30,Albany,US,42.6001,-73.9662,65.17,87,0,4.16,clear sky
31,31,Barra Do Garcas,BR,-15.8900,-52.2567,66.54,48,1,1.34,clear sky
34,34,Arraial Do Cabo,BR,-22.9661,-42.0278,66.34,68,3,12.71,clear sky
38,38,Saint-Philippe,RE,-21.3585,55.7679,72.99,72,39,14.29,scattered clouds


In [20]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [21]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_preferred_cities_df = preferred_cities_df.dropna()

In [22]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Rikitea,PF,72.90,light rain,-23.1203,-134.9692,
8,Jumla,NP,69.17,broken clouds,29.2747,82.1838,
9,Vila Velha,BR,69.75,overcast clouds,-20.3297,-40.2925,
17,Guerrero Negro,MX,70.54,clear sky,27.9769,-114.0611,
19,Fairbanks,US,71.60,broken clouds,64.8378,-147.7164,
27,Isangel,VU,74.57,heavy intensity rain,-19.5500,169.2667,
30,Albany,US,65.17,clear sky,42.6001,-73.9662,
31,Barra Do Garcas,BR,66.54,clear sky,-15.8900,-52.2567,
34,Arraial Do Cabo,BR,66.34,clear sky,-22.9661,-42.0278,
38,Saint-Philippe,RE,72.99,scattered clouds,-21.3585,55.7679,


In [23]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    ## 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search
    hotels = requests.get(base_url, params=params).json()
    try:
         # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    # If an error is experienced, skip the city.
    except:
        print("City Hotel not found. Skipping...")
        pass
  

    # Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")   
        

City Hotel not found. Skipping...
City Hotel not found. Skipping...
City Hotel not found. Skipping...
City Hotel not found. Skipping...
City Hotel not found. Skipping...
City Hotel not found. Skipping...
City Hotel not found. Skipping...
City Hotel not found. Skipping...
City Hotel not found. Skipping...
-----------------------------
Data Retrieval Complete      
-----------------------------


In [25]:
# 7. Drop the rows where there is no Hotel Name.
No_Hotel_value = "NaN"
hotel_df.replace("",No_Hotel_value, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Rikitea,PF,72.90,light rain,-23.1203,-134.9692,People ThankYou
8,Jumla,NP,69.17,broken clouds,29.2747,82.1838,Royal Garden Hotel
9,Vila Velha,BR,69.75,overcast clouds,-20.3297,-40.2925,Hotel Vitória Palace
17,Guerrero Negro,MX,70.54,clear sky,27.9769,-114.0611,"Casa Laguna, Bed & Breakfast"
19,Fairbanks,US,71.60,broken clouds,64.8378,-147.7164,Westmark Fairbanks Hotel & Conference Center
...,...,...,...,...,...,...,...
295,Vanavara,RU,72.79,light rain,60.3400,102.2797,NaN
298,Suchitoto,SV,73.74,overcast clouds,13.9381,-89.0278,Los Almendros de San Lorenzo
302,Ponta Delgada,PT,68.72,few clouds,37.7333,-25.6667,Vila Nova Hotel
311,Leo,BF,73.67,broken clouds,11.1000,-2.1000,Hotel Sissilis


In [26]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [27]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [28]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(50.0, -20.0), zoom_level=2)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure 
fig


Figure(layout=FigureLayout(height='420px'))